In [1]:
import glob
import os
import json
from os import walk
import re
import random
from subprocess import call
import cv2
def image_ann(img_path, image_id, w, h):
    return {'id': image_id,
            'width': w,
            'height': h,
            'file_name': img_path,
            'license': None,
            'flickr_url': '',
            'coco_url': None,
            'date_captured': '2020-12-29T05:36:54Z'}
 
def ann_gen(ann_id, image_id, cat_id, bbox):
    return {'id': ann_id, 'image_id': image_id, 'category_id': cat_id, 'bbox': bbox, 'area': int(bbox[2] * bbox[3] / 100), 'iscrowd': 0}

def json_image(json_path):
    file_core = json_path.split('.')[0]
    vid_class = file_core.split('-')[1]
    core_path = os.path.join(vid_class, file_core)
    files = os.listdir(core_path)    
    indices = range(len(files))
    if len(files) > 2:        
        indices = random.sample(range(len(files)), 3)
    files = [os.path.join(core_path, files[i]) for i in indices]            
    return zip(files, indices)
def generate_ann(total_image_path, breed, bbox, width, height):
    temp = {}
    temp['image_path'] = total_image_path
    temp['breed'] = breed
    temp['bbox'] = bbox
    temp['width'] = width
    temp['height'] = height
    return temp
    
 

In [2]:
# root_dir = '/media/edgar/ybuntu/pet_data/공유_Data_Export'
root_dir = './'
img_dir_prefix = '_frames_fps_5_start_-1.00_end_-1.00_'
result = {}
for (root, dirs, files) in os.walk(root_dir):
    for afile in files:        
        if afile.endswith('json'):
            json_path = os.path.join(root, afile)            
            file_core = afile.split('_')[-1].split('.')[0]
            img_dir = img_dir_prefix + file_core + "_mp4"
            img_path = os.path.join(root, img_dir)
            file_len = len(glob.glob(img_path + "/*.jpg"))             
            with open(json_path) as json_file:
                data = json.load(json_file)
                try:
                    anns = data['annotations']
                    metadata = data['metadata']
                    breed = metadata['animal']['breed']
                    width = metadata['width']
                    height = metadata['height']
                    for ann in anns:
                        frame = ann['frame_number']
                        timestamp = ann['timestamp']                                        
                        img_name = f'frame_{frame}_timestamp_{timestamp}.jpg' 
                        total_image_path = os.path.join(img_path, img_name)
                        if os.path.isfile(total_image_path):            
                            bbox = ann['bounding_box']
                            bbox = [bbox['x'], bbox['y'], bbox['width'], bbox['height']]            
                            if breed not in result.keys(): 
                                result[breed] = []                            
                            result[breed].append(generate_ann(total_image_path[2:], breed, bbox, width, height))
                except:
                    print('something went wrong')
len(result)
          

something went wrong
something went wrong
something went wrong


58

In [3]:
categories = {}
images = []
annotations = []
img_id = 0
category_id = 0
ann_id = 0

def generate_ann_block(cat_list):        
    global test 
    

In [4]:
SAMPLING_RATIO = 0.1

for cat_name in result:
    cat_list = result[cat_name]
    # if len(cat_list) > 1000:
    if len(cat_list) > 10:
        split_point = int(len(result[cat_name]) * SAMPLING_RATIO)
        random.shuffle(cat_list)
        sampled_list = cat_list[:split_point]
        for ann in sampled_list:        
            bbox = ann['bbox']
            breed = ann['breed']
            height = ann["height"]
            width = ann["width"]        
            image_path = ann['image_path']
            if breed not in categories.keys():
                categories[breed] = category_id
                category_id += 1
            images.append(image_ann(image_path, img_id, width, height))            
            annotations.append(ann_gen(ann_id, img_id, categories[breed], bbox))
            img_id +=1
            ann_id +=1

    
    



In [7]:
json_categories = []
for i in categories:
    obj = {}
    obj['id'] = categories[i]
    # obj['name'] = str(categories[i])
    obj['name'] = i
    obj['supercategory'] = 'object'
    json_categories.append(obj)
json_categories



[{'id': 0, 'name': '셔틀랜드 쉽독', 'supercategory': 'object'},
 {'id': 1, 'name': '믹스견', 'supercategory': 'object'},
 {'id': 2, 'name': '포메라니안', 'supercategory': 'object'},
 {'id': 3, 'name': '파피용', 'supercategory': 'object'},
 {'id': 4, 'name': '코리안 숏헤어', 'supercategory': 'object'},
 {'id': 5, 'name': '노르웨이숲', 'supercategory': 'object'},
 {'id': 6, 'name': '푸들', 'supercategory': 'object'},
 {'id': 7, 'name': '스코티쉬 폴드', 'supercategory': 'object'},
 {'id': 8, 'name': '러시안블루', 'supercategory': 'object'},
 {'id': 9, 'name': '브리티쉬 숏헤어', 'supercategory': 'object'},
 {'id': 10, 'name': '미니어처 핀셔', 'supercategory': 'object'},
 {'id': 11, 'name': '치와와', 'supercategory': 'object'},
 {'id': 12, 'name': '말티즈', 'supercategory': 'object'},
 {'id': 13, 'name': '프렌치 불독', 'supercategory': 'object'},
 {'id': 14, 'name': '스코티쉬 폴드 롱헤어', 'supercategory': 'object'},
 {'id': 15, 'name': '닥스훈트', 'supercategory': 'object'},
 {'id': 16, 'name': '비숑 프리제', 'supercategory': 'object'},
 {'id': 17, 'name': '웰시 코기 (펨브록)',

In [8]:
len(images)
len(annotations)

54004

In [10]:
## image_save
# font = cv2.FONT_HERSHEY_SIMPLEX 
# org = (50, 50)
# color = (255, 0, 0)
# nu_image = 100
# for i in range(100):    
#     bbox = annotations[i]['bbox']
#     image_path = images[i]['file_name']
#     img = cv2.imread(image_path)
#     startpoint = (bbox[0], bbox[1])
#     endpoint = (bbox[0] + bbox[2], bbox[1] + bbox[3])
#     image = cv2.rectangle(img, startpoint, endpoint, color, 2)    
#     image = cv2.putText(image, str(bbox), org, font, 2, color, 2, cv2.LINE_AA) 
#     file_core = image_path.split('/')[-2].split('_')[-2] + "_"
#     test_path = 'tester/' + file_core + image_path.split('/')[-1]
#     cv2.imwrite(test_path, image)



In [17]:
json_result = {}
json_result["info"] = {"year": 2021, "date_created": "2020-12-28T07:22:18Z", "version": "1.0", "description": "test None", "contributor": "", "url": "https://app.hasty.ai/projects/fe3692c9-5502-4c07-8ea1-f0e034a9ed02"}
json_result['licenses'] = []
json_result['categories'] = json_categories
json_result['images'] = images
json_result['annotations'] = annotations
with open('breed2.json', 'w') as outfile:
    json.dump(json_result, outfile)


In [10]:
len(annotations)
len(images)

7520

In [85]:
with open('val583_82.txt', 'w') as f:
    for clss in valRaw:
        for item in valRaw[clss]:
            f.write("%s\n" % item)

NameError: name 'valRaw' is not defined

In [25]:
path = "./annotations"
file_list = os.listdir(path)
check_list = []
for json_file in file_list:
    json_path = os.path.join(path, json_file)
    with open(json_path) as json_file:
        data = json.load(json_file)
        file_name = data['file_video'].split('/')[1].split('.')[0]
        class_name = file_name.split('-')[1]
        file_path = os.path.join(class_name, file_name)
        # searching numbering 00000 and add 1 to all the files in the dir
        # if len(glob.glob(file_path + "/*00000.jpg")):
        #     temp_list = sorted(os.listdir(file_path))
        #     for rev in temp_list[::-1]:                
        #         src = os.path.join(file_path, rev)                
        #         num_str = re.findall('[0-9]+', rev)[0]
        #         changed_num = int(num_str) + 1
        #         dst = file_path + "/img_" + str('{:05d}'.format(changed_num)) + '.jpg'
        #         os.rename(src, dst)
        
        # check the number of downloaded file is matched with annotation count
        actual_num_file = len(glob.glob(file_path + "/*jpg"))
        annotation_num_file = len(data['annotations'])
        if actual_num_file is not annotation_num_file:
            cnt = actual_num_file
            cur_data = data['annotations'][cnt - 1:]
            print(file_path)
            for line in cur_data:
                print("{} is miss matched {} : {}".format(file_path, annotation_num_file, actual_num_file))
                url = line["frame_url"]
                current_file_name = file_path + "/img_" + str('{:05d}'.format(cnt)) + '.jpg'                
                r = requests.get(url)
                open(current_file_name, 'wb').write(r.content)    
                print("saved {}".format(current_file_name))
                cnt += 1
            
        check_list.append((annotation_num_file, file_name))
        
        
        # print(len(data['annotations']))
        

tailing/dog-tailing-002676
tailing/dog-tailing-001596
tailing/dog-tailing-001484
feetup/dog-feetup-003034
tailing/dog-tailing-002963
heading/dog-heading-000547
sitdown/cat-sitdown-000009
tailing/dog-tailing-001583
lying/dog-lying-002927
footup/dog-footup-001791
footup/dog-footup-001149
lying/dog-lying-001544
footup/dog-footup-001598
feetup/dog-feetup-001524
bodylower/dog-bodylower-000119
bodylower/dog-bodylower-001749
bodylower/dog-bodylower-002664
lying/dog-lying-000347
feetup/dog-feetup-002924
walkrun/dog-walkrun-002944
sit/dog-sit-000677
footpush/cat-footpush-000231
footup/dog-footup-000597
walkrun/dog-walkrun-003014
sit/dog-sit-000437
heading/dog-heading-000547
lying/dog-lying-000955
bodylower/dog-bodylower-001646


In [15]:
len(check_list)

941

In [167]:
# Object detection Train set
CNT = 10
with open('train10_obj.txt', 'w') as f:
    for clss in trainRaw:
        for item in trainRaw[clss]:
            file_path, frames, clss = item.split(' ')            
            file_list = os.listdir(file_path)
            sampling = random.choices(file_list, k=CNT)
            # print(clss)
            if clss == 'cat':
                clss = '0'                
            else:
                clss = '1'                

            for itemf in sampling:
                f.write("{}\n".format(os.path.join(file_path,itemf, clss)))
            
            # for dir_path in classes[clss]:        
            #     file_list = os.listdir(file_path)
            #     sampling = random.choices(file_list, k=5)
            #     print(sampling)
            #     f.write("%s\n" % item)


In [168]:
# Object detection Val set
with open('val10_obj.txt', 'w') as f:
    for clss in valRaw:
        for item in valRaw[clss]:
            file_path, frames, clss = item.split(' ')            
            file_list = os.listdir(file_path)
            sampling = random.choices(file_list, k=CNT)
            # print(clss)
            if clss == 'cat':
                clss = '0'                
            else:
                clss = '1'                

            for itemf in sampling:
                f.write("{}\n".format(os.path.join(file_path,itemf, clss)))